# Neurosymbolic Software Tutorial - NEAR Bouncing Ball

<a target="_blank" href="https://colab.research.google.com/github/kavigupta/neurosym-lib/blob/main/tutorial/bouncing_ball_exercise_skeleton.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Instruction
- Navigating this notebook on Google Colab: There will be text blocks and code blocks throughout the notebook. The text blocks, such as this one, will contain instructions and questions for you to consider. The code blocks, such as the one below, will contain executible code. Sometimes you will have to modify the code blocks following the instructions in the text blocks. You can run the code block by either pressing control/cmd + enter or by clicking the arrow on left-hand side.
- Saving Work: If you wish to save your work in this .ipynb, we recommend downloading the compressed repository from GitHub, unzipping it, uploading it to Google Drive, and opening this notebook from within Google Drive.

## Notebook

In this notebook, you will construct a DSL to simulate a bouncing ball

In [1]:
%load_ext autoreload
%load_ext jupyter_black
%autoreload 2
%matplotlib inline

import logging
import os
import itertools
    
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import neurosym as ns
from neurosym.examples import near

pl = ns.import_pytorch_lightning()


In [2]:
torch.manual_seed(0)
np.random.seed(0)

## Data

We then load and plot some bouncing ball trajectories. Note that these trajectories are represented as a list `[x, y, vx, vy]`

In [3]:
dataset_factory = lambda train_seed: ns.DatasetWrapper(
    ns.DatasetFromNpy(
        f"bouncing_ball_exercise/data/bounce_example/train_ex_data.npy",
        f"bouncing_ball_exercise/data/bounce_example/train_ex_labels.npy",
        train_seed,
    ),
    ns.DatasetFromNpy(
        f"bouncing_ball_exercise/data/bounce_example/test_ex_data.npy",
        f"bouncing_ball_exercise/data/bounce_example/test_ex_labels.npy",
        None,
    ),
    batch_size=200,
)
datamodule = dataset_factory(42)

In [4]:
def plot_trajectory(trajectory, color, label=None):
    plt.scatter(
        trajectory[:, 0], trajectory[:, 1], marker="o", color=color, label=label
    )
    plt.plot(trajectory[:, 0], trajectory[:, 1], alpha=0.2, color=color)
    plt.xlim(-5, 10)
    plt.ylim(-5, 7)
    plt.grid(True)

In [5]:
for i in range(3):
    plot_trajectory(datamodule.train.inputs[i], f"C{i}")

In [6]:
print("input[0] :", datamodule.train.inputs[i, 0])
print("output[0]:", datamodule.train.outputs[i, 0])

## Exercise: DSL

Fill in the `bounce_dsl` to parameterize the space of functions that could represent the trajectories of bouncing balls.

In [7]:
L = 4


def bounce_dsl():
    dslf = ns.DSLFactory(L=L, max_overall_depth=5)
    "YOUR CODE HERE"
    dslf.prune_to("[$fL] -> [$fL]")
    return dslf.finalize()

In [8]:
dsl = bounce_dsl()

### DSL Printout

See your DSL printed below, and ensure it is what you would expect

In [9]:
print(dsl.render())

### Setting up Neural DSL

In [10]:
input_dim, output_dim = 4, 4
max_depth = 16

### Run NEAR

In [11]:
t = ns.TypeDefiner(L=input_dim, O=output_dim)
t.typedef("fL", "{f, $L}")

In [12]:
neural_dsl = near.NeuralDSL.from_dsl(
    dsl, neural_hole_filler=near.GenericMLPRNNNeuralHoleFiller(hidden_size=10)
)
g = near.validated_near_graph(
    neural_dsl,
    t("([{f, $L}]) -> [{f, $O}]"),
    is_goal=lambda x: True,
    cost=near.ValidationCost(
        trainer_cfg=near.NEARTrainerConfig(
            lr=0.1,
            n_epochs=100,
            accelerator="cpu",
            loss_callback=nn.functional.mse_loss,
        ),
        neural_dsl=neural_dsl,
        datamodule=datamodule,
        progress_by_epoch=True,
    ),
    validation_epochs=4000,
)

In [13]:
best_programs = list(
    itertools.islice(
        ns.search.bounded_astar(g, max_depth=1000, max_iterations=10000), 3
    )
)

### Top 3 Programs

Now, we show the top 3 programs

In [15]:
for prog in best_programs:
    print(ns.render_s_expression(prog.uninitialize()))

## Plotting Trajectories

We plot all the trajectories

In [16]:
def program_to_trajectory(dsl, best_program):
    T = 100
    path = np.zeros((T, 4))
    X = torch.tensor(
        np.array([0.21413583, 4.4062634, 3.4344807, 0.12440437]), dtype=torch.float32
    )
    for t in range(T):
        path[t, :] = X.detach().numpy()
        Y = dsl.compute(best_program)(X.unsqueeze(0)).squeeze(0)
        X = Y
    return path


def plot_programs(dsl, best_programs):
    plt.figure(figsize=(8, 8))

    plot_trajectory(datamodule.train.inputs[0], "black")
    for i, (prog) in enumerate(best_programs):
        plot_trajectory(
            program_to_trajectory(dsl, prog),
            f"C{i}",
            label=ns.render_s_expression(prog.uninitialize()).replace("$", r"\$"),
        )

    plt.title("Bouncing ball (ground truth in black)")
    plt.legend()
    plt.show()

In [17]:
plot_programs(dsl, best_programs)